In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### EXPLORATORY DATA ANALYSIS

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train.csv')

In [ ]:
# HOW TO AGGREGATE THE DATA
aggregate_by = 'median'

In [ ]:
print(f'Shape of the train-set = {df.shape}')
print()
display(df.head(10))
print()
display(df.describe())

Shape of the train-set = (1246837, 25)



,UNIQUE_IDENTIFIER,SEQUENCE_NO,STATUS_CHECK,CATEGORY_1,CATEGORY_2,ACTIVE_YN,ENTRY,REVENUE,WINNINGS_1,WINNINGS_2,DISCOUNT,DEPOSIT,DEPOSIT_NUMBER,DEPOSIT_2,WITHDRAW,WITHDRAW_NUMBER,DEPOSIT_TRAILS,ENTRY_NUMBER,WINNINGS_NUMBER,PRACTICE_ENTRY,PRACTICE_WINNINGS,PRACTICE_ENTRY_NUMBER,PRACTICE_WINNINGS_NUMBER,Y1,Y2
0,98481267304,1,0,M,B,1,0.00000,0.000000,0.00000,0.0,0.0,0.25,0.1,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.449287,101.369863
1,98481267304,2,0,M,B,1,0.13735,0.011550,0.09990,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,1.8,1.0,0.0,0.0,0.0,0.0,4.449287,101.369863
2,98481267304,3,0,M,B,1,0.15835,0.010425,0.02420,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.7,0.3,0.0,0.0,0.0,0.0,4.449287,101.369863
3,98481267304,4,0,M,B,1,0.44490,0.035850,0.23980,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,2.8,1.5,0.0,0.0,0.0,0.0,4.449287,101.369863
4,98481267304,5,0,M,B,1,0.00000,0.000000,0.00000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.449287,101.369863
5,98481267304,6,0,M,B,1,0.00000,0.000000,0.00000,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,4.449287,101.369863
6,98481267304,7,0,M,B,1,0.04505,0.002950,0.00660,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,1.1,0.3,0.0,0.0,0.0,0.0,4.449287,101.369863
7,98481267698,1,1,M,I,1,5.00000,0.750000,0.00000,0.0,1.0,10.00,0.1,10.00,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,7.182779,115.068493
8,98481267698,2,1,M,I,1,11.25000,1.690000,24.64500,0.0,1.4,0.00,0.0,0.00,0.0,0.0,0.0,0.7,0.5,0.0,0.0,0.0,0.0,7.182779,115.068493
9,98481267698,3,1,M,I,1,47.14050,6.909500,45.69765,0.0,0.0,10.00,0.1,0.00,0.0,0.0,2.0,1.8,0.8,0.0,0.0,0.0,0.0,7.182779,115.068493


,UNIQUE_IDENTIFIER,SEQUENCE_NO,STATUS_CHECK,ACTIVE_YN,ENTRY,REVENUE,WINNINGS_1,WINNINGS_2,DISCOUNT,DEPOSIT,DEPOSIT_NUMBER,DEPOSIT_2,WITHDRAW,WITHDRAW_NUMBER,DEPOSIT_TRAILS,ENTRY_NUMBER,WINNINGS_NUMBER,PRACTICE_ENTRY,PRACTICE_WINNINGS,PRACTICE_ENTRY_NUMBER,PRACTICE_WINNINGS_NUMBER,Y1,Y2
count,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06,1.246837e+06
mean,9.852168e+10,1.105455e+01,4.223696e-01,9.868451e-01,4.726720e+00,5.350111e-01,3.700804e+00,4.262171e-02,4.482682e-01,7.602949e-01,3.938229e-02,1.011141e-01,6.076243e-02,1.604540e-03,1.379019e-01,2.268259e+00,7.415130e-01,3.447181e-01,2.749940e-01,3.990834e-02,1.652854e-02,6.508164e+00,2.081873e+02
std,6.339280e+06,7.850787e+00,4.939370e-01,1.139379e-01,2.609701e+01,2.557525e+00,2.222386e+01,8.355851e-01,2.193891e+00,3.407900e+00,7.926851e-02,8.268497e-01,1.133188e+00,1.532719e-02,1.171732e+00,4.263824e+00,1.567570e+00,5.121462e+00,4.754888e+00,4.349036e-01,1.926375e-01,1.162251e+01,2.269953e+02
min,9.848127e+10,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.604128e+00,0.000000e+00
25%,9.851986e+10,4.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.012769e+00,3.835616e+01
50%,9.852279e+10,1.000000e+01,0.000000e+00,1.000000e+00,4.500000e-01,5.970000e-02,1.888000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e-01,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.202048e+00,1.150685e+02
75%,9.852542e+10,1.700000e+01,1.000000e+00,1.000000e+00,1.962600e+00,2.637500e-01,1.276900e+00,0.000000e+00,0.000000e+00,5.000000e-01,1.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.300000e+00,8.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.042758e+00,3.068493e+02
max,9.852732e+10,3.000000e+01,1.000000e+00,1.000000e+00,2.819799e+03,2.096527e+02,2.086580e+03,3.531417e+02,2.000000e+02,1.250000e+02,3.000000e+00,2.500000e+01,2.400000e+02,1.300000e+00,6.450000e+02,1.077000e+02,4.510000e+01,6.140000e+02,6.732000e+02,3.330000e+01,1.870000e+01,8.340632e+02,1.000000e+03


In [ ]:
!pip install pandas-profiling
#!pip install sweetviz
#!pip install autoviz
#!pip install dtale

##### 1. PANDAS-PROFILING

### TESTING ML MODELS

#### 1. XGBOOST

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
#from sklearn.grid_search import GridSearchCV   #Perforing grid search

In [ ]:
# TAKING THE OBJECT TYPE COLUMNS ONLY
cat_columns = df.select_dtypes(['object']).columns

In [ ]:
# PRINT THE OBJECT TYPE COLUMNS
cat_columns

Index(['CATEGORY_1', 'CATEGORY_2'], dtype='object')

In [ ]:
# CONVERT OBJECT TO CATEGORY
for column in ['CATEGORY_1', 'CATEGORY_2']:
    df[column] = df[column].astype('category')

In [ ]:
# CONVERT CATEGORY TO INT8
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

In [ ]:
# INT8 TO INT 64
for column in ['CATEGORY_1', 'CATEGORY_2']:
    df[column] = df[column].astype('int64')

In [ ]:
# RESULTS AFTER CONVERTSION TO INT64
print('CATEGORY 1:')
print(np.unique(np.array(df['CATEGORY_1']), return_counts = True))
print()
print('CATEGORY 2:')
print(np.unique(np.array(df['CATEGORY_2']), return_counts = True))

CATEGORY 1:
(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), array([ 76083, 651499, 111840,   1879,  18572,   1805,   5793, 309891,
         1247,  68228]))

CATEGORY 2:
(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([146104, 117569,  18152, 257078,   6568,  21381,   3613,   7011,
       669361]))


In [ ]:
# CHECKING FOR PROPER CONVERSION OF OBJECT TO INT64
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1246837 entries, 0 to 1246836
Data columns (total 25 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   UNIQUE_IDENTIFIER         1246837 non-null  int64  
 1   SEQUENCE_NO               1246837 non-null  int64  
 2   STATUS_CHECK              1246837 non-null  int64  
 3   CATEGORY_1                1246837 non-null  int64  
 4   CATEGORY_2                1246837 non-null  int64  
 5   ACTIVE_YN                 1246837 non-null  int64  
 6   ENTRY                     1246837 non-null  float64
 7   REVENUE                   1246837 non-null  float64
 8   WINNINGS_1                1246837 non-null  float64
 9   WINNINGS_2                1246837 non-null  float64
 10  DISCOUNT                  1246837 non-null  float64
 11  DEPOSIT                   1246837 non-null  float64
 12  DEPOSIT_NUMBER            1246837 non-null  float64
 13  DEPOSIT_2                 1

In [ ]:
print(np.unique(np.array(df['UNIQUE_IDENTIFIER']), return_counts = True)[0].shape)

(96298,)


In [ ]:
# AGGREGATING THE DUPLICATE ROWS
df = df.groupby(['UNIQUE_IDENTIFIER'],as_index = False).agg(aggregate_by)
df.shape

(96298, 25)

In [ ]:
#col_to_drop = ['ENTRY', 'WINNINGS_1', 'ENTRY_NUMBER','PRACTICE_ENTRY','PRACTICE_ENTRY_NUMBER','CATEGORY_1','CATEGORY_2']
#df = df.drop(col_to_drop, axis = 1)
#col_to_take = ['SEQUENCE_NO', 'STATUS_CHECK', 'ENTRY', 'REVENUE', 'WINNINGS_1', 'DEPOSIT','Y1', 'Y2']
#df = df.loc[:,col_to_take]

In [ ]:
df.head()

,UNIQUE_IDENTIFIER,SEQUENCE_NO,STATUS_CHECK,CATEGORY_1,CATEGORY_2,ACTIVE_YN,ENTRY,REVENUE,WINNINGS_1,WINNINGS_2,DISCOUNT,DEPOSIT,DEPOSIT_NUMBER,DEPOSIT_2,WITHDRAW,WITHDRAW_NUMBER,DEPOSIT_TRAILS,ENTRY_NUMBER,WINNINGS_NUMBER,PRACTICE_ENTRY,PRACTICE_WINNINGS,PRACTICE_ENTRY_NUMBER,PRACTICE_WINNINGS_NUMBER,Y1,Y2
0,98481267304,4.0,0.0,9.0,1.0,1.0,0.04505,0.00295,0.00660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.3,0.0,0.0,0.0,0.0,4.449287,101.369863
1,98481267698,10.0,1.0,9.0,8.0,1.0,37.50000,4.57500,18.15835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.1,0.0,0.0,0.0,0.0,7.182779,115.068493
2,98481269325,9.5,0.0,9.0,3.0,1.0,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,3.977371,24.657534
3,98481271512,3.0,0.0,9.0,4.0,1.0,0.20000,0.03000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.3,0.0,0.0,0.0,0.0,0.0,3.988122,10.958904
4,98481273023,1.0,0.0,9.0,8.0,1.0,0.50000,0.08000,0.00000,0.0,0.0,0.5,0.1,0.5,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,4.128875,46.575342


In [ ]:
# INPUT OUTPUT SPLIT
X, y = df.iloc[:,:-2],df.iloc[:,-2:]

In [ ]:
# CONVERSION TO DATAMATRIX FOR FASTER IMPLEMENTATION
#data_dmatrix = xgb.DMatrix(data=X, label = y, enable_categorical=True)

In [ ]:
# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# CONVERSION TO DATAMATRIX FOR FASTER IMPLEMENTATION
#dtrain = xgb.DMatrix(X_train, label=y_train)
#dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
# PRINT SHAPES OF TRAIN AND TEST DATA
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(64519, 23)
(31779, 23)
(64519, 2)
(31779, 2)


In [ ]:
'''
# PARAMS TO TUNE
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:squarederror',
}
params['eval_metric'] = "mae"
num_boost_round = 999
'''


'\n# PARAMS TO TUNE\nparams = {\n    # Parameters that we are going to tune.\n    \'max_depth\':6,\n    \'min_child_weight\': 1,\n    \'eta\':.3,\n    \'subsample\': 1,\n    \'colsample_bytree\': 1,\n    # Other parameters\n    \'objective\':\'reg:squarederror\',\n}\nparams[\'eval_metric\'] = "mae"\nnum_boost_round = 999\n'

In [ ]:
'''
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)
'''

'\nmodel = xgb.train(\n    params,\n    dtrain,\n    num_boost_round=num_boost_round,\n    evals=[(dtest, "Test")],\n    early_stopping_rounds=10\n)\n'

In [ ]:
'''print("Best MAE: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))
'''

'print("Best MAE: {:.2f} with {} rounds".format(\n                 model.best_score,\n                 model.best_iteration+1))\n'

In [ ]:
'''
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)

display(cv_results)
cv_results['test-mae-mean'].min()
'''

"\ncv_results = xgb.cv(\n    params,\n    dtrain,\n    num_boost_round=num_boost_round,\n    seed=42,\n    nfold=5,\n    metrics={'mae'},\n    early_stopping_rounds=10\n)\n\ndisplay(cv_results)\ncv_results['test-mae-mean'].min()\n"

In [ ]:
'''
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(5,12)
    for min_child_weight in range(3,8)
]
'''

'\ngridsearch_params = [\n    (max_depth, min_child_weight)\n    for max_depth in range(5,12)\n    for min_child_weight in range(3,8)\n]\n'

In [ ]:
'''
# MAXDEPTH AND MINCHILDWEIGHT
# Define initial best params and MAE
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))
'''

'\n# MAXDEPTH AND MINCHILDWEIGHT\n# Define initial best params and MAE\nmin_mae = float("Inf")\nbest_params = None\nfor max_depth, min_child_weight in gridsearch_params:\n    print("CV with max_depth={}, min_child_weight={}".format(\n                             max_depth,\n                             min_child_weight))\n    # Update our parameters\n    params[\'max_depth\'] = max_depth\n    params[\'min_child_weight\'] = min_child_weight\n    # Run CV\n    cv_results = xgb.cv(\n        params,\n        dtrain,\n        num_boost_round=num_boost_round,\n        seed=42,\n        nfold=5,\n        metrics={\'mae\'},\n        early_stopping_rounds=10\n    )\n    # Update best MAE\n    mean_mae = cv_results[\'test-mae-mean\'].min()\n    boost_rounds = cv_results[\'test-mae-mean\'].argmin()\n    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))\n    if mean_mae < min_mae:\n        min_mae = mean_mae\n        best_params = (max_depth,min_child_weight)\nprint("Best params: {}, 

In [ ]:
'''
params['max_depth'] = best_params[0]
params['min_child_weight'] = best_params[1]
params
'''

"\nparams['max_depth'] = best_params[0]\nparams['min_child_weight'] = best_params[1]\nparams\n"

In [ ]:
# subsample and colsample_bytree
'''
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

params['subsample'] = best_params[0]
params['colsample_bytree'] = best_params[1]
'''

'\ngridsearch_params = [\n    (subsample, colsample)\n    for subsample in [i/10. for i in range(7,11)]\n    for colsample in [i/10. for i in range(7,11)]\n]\n\nmin_mae = float("Inf")\nbest_params = None\n# We start by the largest values and go down to the smallest\nfor subsample, colsample in reversed(gridsearch_params):\n    print("CV with subsample={}, colsample={}".format(\n                             subsample,\n                             colsample))\n    # We update our parameters\n    params[\'subsample\'] = subsample\n    params[\'colsample_bytree\'] = colsample\n    # Run CV\n    cv_results = xgb.cv(\n        params,\n        dtrain,\n        num_boost_round=num_boost_round,\n        seed=42,\n        nfold=5,\n        metrics={\'mae\'},\n        early_stopping_rounds=10\n    )\n    # Update best score\n    mean_mae = cv_results[\'test-mae-mean\'].min()\n    boost_rounds = cv_results[\'test-mae-mean\'].argmin()\n    print("\tMAE {} for {} rounds".format(mean_mae, boost_roun

In [ ]:
# LEARNING RATE
'''
%time
# This can take some time…
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    %time 
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10)
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

params['eta'] = best_params
'''

'\n%time\n# This can take some time…\nmin_mae = float("Inf")\nbest_params = None\nfor eta in [.3, .2, .1, .05, .01, .005]:\n    print("CV with eta={}".format(eta))\n    # We update our parameters\n    params[\'eta\'] = eta\n    # Run and time CV\n    %time \n    cv_results = xgb.cv(\n            params,\n            dtrain,\n            num_boost_round=num_boost_round,\n            seed=42,\n            nfold=5,\n            metrics=[\'mae\'],\n            early_stopping_rounds=10)\n    # Update best score\n    mean_mae = cv_results[\'test-mae-mean\'].min()\n    boost_rounds = cv_results[\'test-mae-mean\'].argmin()\n    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))\n    if mean_mae < min_mae:\n        min_mae = mean_mae\n        best_params = eta\nprint("Best params: {}, MAE: {}".format(best_params, min_mae))\n\nparams[\'eta\'] = best_params\n'

In [ ]:
'''
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best MAE: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))

print(f'Parameters : {params}')
'''


'\nmodel = xgb.train(\n    params,\n    dtrain,\n    num_boost_round=num_boost_round,\n    evals=[(dtest, "Test")],\n    early_stopping_rounds=10\n)\n\nprint("Best MAE: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1))\n\nprint(f\'Parameters : {params}\')\n'

In [ ]:
#model.best_iteration + 1

In [ ]:
#preds = model.predict(dtest)

In [ ]:
# DEFINE THE MODEL

#xg_reg = MultiOutputRegressor(xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.01,
#               max_depth = 5, alpha = 10, n_estimators = 1000))
xg_reg = MultiOutputRegressor(xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.5, learning_rate = 0.01,
                max_depth = 5, alpha = 10, n_estimators = 1100, tree_method = 'hist',
                          min_child_weight = 5, eval_metric = 'rmse'))

In [ ]:
# FITTING THE MODEL
xg_reg.fit(X_train,y_train)#, early_stopping_rounds=5, eval_set = [(X_test, y_test)])

MultiOutputRegressor(estimator=XGBRegressor(alpha=10, base_score=0.5,
                                            booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=0.5,
                                            eval_metric='rmse', gamma=0,
                                            importance_type='gain',
                                            learning_rate=0.01,
                                            max_delta_step=0, max_depth=5,
                                            min_child_weight=5, missing=None,
                                            n_estimators=1100, n_jobs=1,
                                            nthread=None,
                                            objective='reg:squarederror',
                                            random_state=0, reg_alpha=0,
                                     

In [ ]:
# PREDICTIONS FOR TEST SET
preds = xg_reg.predict(X_test)

In [ ]:
preds

array([[  5.097913 ,  95.367744 ],
       [  3.8246875,   8.998213 ],
       [  4.2425013,  38.408115 ],
       ...,
       [  4.1444554,  60.95529  ],
       [  5.6602335, 415.64194  ],
       [  4.0442114,  22.215275 ]], dtype=float32)

In [ ]:
y_test - preds

,Y1,Y2
34721,-0.402448,-46.052676
38544,0.161001,-3.518761
127,-0.270171,-8.271129
68980,0.019846,0.488968
26735,-1.324368,-51.353913
...,...,...
41674,-0.162192,-10.063034
13717,-1.805286,-129.189447
39556,-0.168872,-44.516936
42072,-1.467981,104.906008


In [ ]:
y_test.describe()

,Y1,Y2
count,31779.000000,31779.000000
mean,5.395905,114.698041
std,8.440347,185.897186
min,2.604128,0.000000
25%,3.984064,8.219178
50%,4.017446,27.397260
75%,4.327899,128.767123
max,398.327544,1000.000000



Mean(Y1, Y2) || Std(Y1, Y2)

'LAST' : [ 3.6545959 74.96868  ]
[ 1.614602 47.770298]
'MEAN' : [  5.3569307 115.524765 ]
[  5.063568 133.9396  ]

In [ ]:
print(preds.mean(axis = 0))
print(preds.std(axis = 0))

[  5.406557 114.19127 ]
[  5.8360677 137.2647   ]


In [ ]:
# CALCULATING THE RMSE ERROR
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 87.380185


### CREATING THE CSV SUBMISSION FILE

In [ ]:
# LOADING THE TEST FILE
df_test = pd.read_csv('/content/drive/MyDrive/test.csv')
print(f'Shape of the test-set = {df_test.shape}')
print()
display(df_test.head(10))
print()
display(df_test.describe())

Shape of the test-set = (979059, 23)



,UNIQUE_IDENTIFIER,SEQUENCE_NO,STATUS_CHECK,CATEGORY_1,CATEGORY_2,ACTIVE_YN,ENTRY,REVENUE,WINNINGS_1,WINNINGS_2,DISCOUNT,DEPOSIT,DEPOSIT_NUMBER,DEPOSIT_2,WITHDRAW,WITHDRAW_NUMBER,DEPOSIT_TRAILS,ENTRY_NUMBER,WINNINGS_NUMBER,PRACTICE_ENTRY,PRACTICE_WINNINGS,PRACTICE_ENTRY_NUMBER,PRACTICE_WINNINGS_NUMBER
0,98481269471,1,0,M,D,1,5.975,0.90500,0.4100,0.0,2.0,5.5,0.3,1.0,0.0,0.0,9.8,1.4,0.2,0.000000,0.0000,0.0,0.0
1,98481269471,2,0,M,D,0,0.000,0.00000,0.0000,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0
2,98481270499,1,1,M,D,1,0.050,0.00750,0.0850,0.0,0.0,1.0,0.1,1.0,0.0,0.0,0.0,0.1,0.1,0.000000,0.0000,0.0,0.0
3,98481270499,2,1,M,D,1,1.750,0.26250,1.0200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,1.2,0.000000,0.0000,0.0,0.0
4,98481272694,1,0,M,C,1,0.125,0.01875,0.0425,0.0,4.0,10.0,0.1,10.0,0.0,0.0,0.0,0.5,0.1,0.000000,0.0000,0.0,0.0
5,98481272694,2,0,M,C,1,0.000,0.00000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.023800,2.4992,3.4,0.8
6,98481272694,3,0,M,C,1,0.000,0.00000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.824340,7.8232,6.3,2.1
7,98481272694,4,0,M,C,1,0.000,0.00000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.809267,1.7952,3.1,0.4
8,98481272694,5,0,M,C,1,0.000,0.00000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0000,0.1,0.0
9,98481272694,6,0,M,C,1,0.000,0.00000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.136980,2.8160,2.7,0.9


,UNIQUE_IDENTIFIER,SEQUENCE_NO,STATUS_CHECK,ACTIVE_YN,ENTRY,REVENUE,WINNINGS_1,WINNINGS_2,DISCOUNT,DEPOSIT,DEPOSIT_NUMBER,DEPOSIT_2,WITHDRAW,WITHDRAW_NUMBER,DEPOSIT_TRAILS,ENTRY_NUMBER,WINNINGS_NUMBER,PRACTICE_ENTRY,PRACTICE_WINNINGS,PRACTICE_ENTRY_NUMBER,PRACTICE_WINNINGS_NUMBER
count,9.790590e+05,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000,979059.000000
mean,9.853018e+10,11.440345,0.394211,0.969687,3.172277,0.366817,2.468117,0.082370,0.411340,0.540256,0.029050,0.069796,0.061635,0.002285,0.127763,1.851157,0.617564,0.209117,0.164661,0.033872,0.015349
std,8.934826e+06,7.825283,0.488681,0.171447,21.375312,2.150099,18.271798,1.316261,2.158291,3.442908,0.067829,0.659688,1.085939,0.017874,1.057838,3.692403,1.402774,3.838995,3.615380,0.371677,0.167900
min,9.848127e+10,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.852772e+10,5.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9.852932e+10,10.000000,0.000000,1.000000,0.275000,0.037500,0.113300,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.600000,0.100000,0.000000,0.000000,0.000000,0.000000
75%,9.853655e+10,17.000000,1.000000,1.000000,1.300000,0.177347,0.850550,0.000000,0.180000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.900000,0.600000,0.000000,0.000000,0.000000,0.000000
max,9.853721e+10,30.000000,1.000000,1.000000,2386.920000,245.500000,2201.752600,500.000000,210.000000,210.000000,2.400000,50.000000,160.000000,1.000000,250.000000,93.300000,49.500000,736.830000,665.866670,31.100000,12.500000


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 979059 entries, 0 to 979058
Data columns (total 23 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   UNIQUE_IDENTIFIER         979059 non-null  int64  
 1   SEQUENCE_NO               979059 non-null  int64  
 2   STATUS_CHECK              979059 non-null  int64  
 3   CATEGORY_1                979059 non-null  object 
 4   CATEGORY_2                979059 non-null  object 
 5   ACTIVE_YN                 979059 non-null  int64  
 6   ENTRY                     979059 non-null  float64
 7   REVENUE                   979059 non-null  float64
 8   WINNINGS_1                979059 non-null  float64
 9   WINNINGS_2                979059 non-null  float64
 10  DISCOUNT                  979059 non-null  float64
 11  DEPOSIT                   979059 non-null  float64
 12  DEPOSIT_NUMBER            979059 non-null  float64
 13  DEPOSIT_2                 979059 non-null  f

In [ ]:
# SELECTING THE OBJECT COLUMNS
cat_columns_test = df_test.select_dtypes(['object']).columns

In [ ]:
cat_columns_test

Index(['CATEGORY_1', 'CATEGORY_2'], dtype='object')

In [ ]:
# CONVERT OBJECT TO CATEGORY
for column in ['CATEGORY_1', 'CATEGORY_2']:
    df_test[column] = df_test[column].astype('category')

In [ ]:
# CONVERT CATEGORY TO INT8
df_test[cat_columns] = df_test[cat_columns].apply(lambda x: x.cat.codes)

In [ ]:
# INT8 TO INT 64
for column in ['CATEGORY_1', 'CATEGORY_2']:
    df_test[column] = df_test[column].astype('int64')

In [ ]:
# RESULTS AFTER CONVERTSION TO INT64
print('CATEGORY 1:')
print(np.unique(np.array(df_test['CATEGORY_1']), return_counts = True))
print()
print('CATEGORY 2:')
print(np.unique(np.array(df_test['CATEGORY_2']), return_counts = True))

CATEGORY 1:
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]), array([ 69006, 399083,  60921,  38194, 117953,   1908,  12300,    720,
        10046, 232404,   1068,  35456]))

CATEGORY 2:
(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([184475,  80352,  30172, 374768,   6643,  19865,   4929,  16978,
       260877]))


In [ ]:
# CONVERSION TO DATAMATRIX FOR FASTER IMPLEMENTATION
#data_dmatrix = xgb.DMatrix(data=X, label=y, enable_categorical=True)
print(np.unique(np.array(df_test['UNIQUE_IDENTIFIER']), return_counts = True)[0].shape)

(65242,)


In [ ]:
df_test = df_test.groupby(['UNIQUE_IDENTIFIER'],as_index = False).agg(aggregate_by)
df_test.shape

(65242, 23)

In [ ]:
#df_test.drop(col_to_drop, axis = 1)
df_test.shape

(65242, 23)

In [ ]:
#col_to_take = ['SEQUENCE_NO', 'STATUS_CHECK', 'ENTRY', 'REVENUE', 'WINNINGS_1', 'DEPOSIT']
#df_test = df_test.loc[:,col_to_take]

In [ ]:
# PREDICTIONS FOR TEST SET
pred_test = xg_reg.predict(df_test)

In [ ]:
pred_test[0]

array([3.9512813, 7.483494 ], dtype=float32)

In [ ]:
# SAMPLE SUBMISSION FILE

sample = pd.read_csv('/content/drive/MyDrive/sample_submission_random.csv')
print(f'Shape of the train-set = {sample.shape}')
print()
display(sample.head(10))

Shape of the train-set = (65242, 3)



,UNIQUE_IDENTIFIER,Y1,Y2
0,98481269471,1.682706,162.035289
1,98481270499,9.134302,4.972718
2,98481272694,3.241823,119.685833
3,98481273365,5.206650,186.116324
4,98481278321,1.901634,107.374648
5,98481278505,7.499764,56.777380
6,98481280559,4.795677,74.851192
7,98481280993,4.123684,169.927371
8,98481285757,6.091130,109.360907
9,98481288648,0.105779,121.248585


In [ ]:
print(df_test['UNIQUE_IDENTIFIER'].shape)
print('Unique count:')
np.unique(np.array(df_test['UNIQUE_IDENTIFIER']), return_counts = True)[0].shape
# SO THERE SO MANY DUPLICATE CUSTOMER (UNIQUE_IDENTIFIER) IN test.csv but I have aggregated them

(65242,)
Unique count:


(65242,)

In [ ]:
y1_pred = pred_test[:,0]
y2_pred = pred_test[:,1]
#y2_pred = pred_test[:,0]


In [ ]:
dict = {'UNIQUE_IDENTIFIER': df_test['UNIQUE_IDENTIFIER'] ,'Y1': y1_pred, 'Y2' : y2_pred}

In [ ]:
my_submission = pd.DataFrame(dict,index = None)

In [ ]:
my_submission.head()

,UNIQUE_IDENTIFIER,Y1,Y2
0,98481269471,3.951281,7.483494
1,98481270499,4.817670,42.932449
2,98481272694,4.229680,31.685501
3,98481273365,4.247539,191.429001
4,98481278321,3.878090,12.145032


In [ ]:
my_submission.shape

(65242, 3)

In [ ]:
#my_submission = my_submission.drop_duplicates(subset = ["UNIQUE_IDENTIFIER"])
#my_submission.shape

In [ ]:
my_submission.to_csv('sub9044.csv', index=False)

__notebook__.ipynb  submission9.csv


In [ ]:
%cd .. dfdf


[Errno 2] No such file or directory: '.. dfdf'
/content
